In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
pizzahutfilepath = r"C:\Users\Acer\Desktop\nmtu_class 3\Class 3 _ Pizzahut - RFM\De bai\Round 2 - Data set.xlsx"
pizzahut = pd.read_excel(pizzahutfilepath,sheet_name = 'Purchasing data')
pizzahut.head()

,Unnamed: 0,BillID,Channel,OrderFrom,TransactionDate,SalesAmount,CustomerID,CustomerGender,VoucherStatus,Province
0,0,0,Take Away,CALL CENTER,2021-10-01 00:00:00+00:00,296891,1753863,Unknown,No,Ho Chi Minh City
1,1,1,Take Away,STORE,2021-10-01 00:00:00+00:00,301782,1124050,Unknown,No,Hanoi
2,2,2,Take Away,WEBSITE,2021-10-01 00:00:00+00:00,319792,1626827,Male,No,Hanoi
3,3,3,Take Away,STORE,2021-10-01 00:00:00+00:00,424762,125643,Male,No,Hanoi
4,4,4,Delivery,STORE,2021-10-01 00:00:00+00:00,280031,2117237,Unknown,No,Hanoi


In [3]:
# check data type
print(pizzahut.dtypes)

Unnamed: 0          int64
BillID              int64
Channel            object
OrderFrom          object
TransactionDate    object
SalesAmount         int64
CustomerID          int64
CustomerGender     object
VoucherStatus      object
Province           object
dtype: object


In [4]:
# change type
pizzahut = pizzahut.astype({'Channel': 'string', 'OrderFrom': 'string',  'CustomerGender': 'string', 'VoucherStatus': 'string', 'Province': 'string'})
pizzahut['TransactionDate'] = pd.to_datetime(pizzahut['TransactionDate'])
print(pizzahut.dtypes)

Unnamed: 0                       int64
BillID                           int64
Channel                         string
OrderFrom                       string
TransactionDate    datetime64[ns, UTC]
SalesAmount                      int64
CustomerID                       int64
CustomerGender                  string
VoucherStatus                   string
Province                        string
dtype: object


In [5]:
# lastest date
lastest_date = pizzahut['TransactionDate'].max()
# target date is one day after last transaction date
target_date = lastest_date + pd.Timedelta(days=1)
target_date

Timestamp('2023-01-10 00:00:00+0000', tz='UTC')

In [6]:
# quartile bins
qbin = 4

In [7]:
# recency df
recency = pizzahut.groupby('CustomerID')['TransactionDate'].agg('max').reset_index()
recency.columns = ['CustomerID', 'LatestTransactionDate']
recency.head()

,CustomerID,LatestTransactionDate
0,0,2021-12-24 00:00:00+00:00
1,1,2022-03-07 00:00:00+00:00
2,8,2022-07-11 00:00:00+00:00
3,16,2022-09-16 00:00:00+00:00
4,18,2022-01-29 00:00:00+00:00


In [8]:
# days from last transaction
recency['DaysSinceLastTransaction'] = (target_date - recency['LatestTransactionDate']).dt.days
recency.head()

,CustomerID,LatestTransactionDate,DaysSinceLastTransaction
0,0,2021-12-24 00:00:00+00:00,382
1,1,2022-03-07 00:00:00+00:00,309
2,8,2022-07-11 00:00:00+00:00,183
3,16,2022-09-16 00:00:00+00:00,116
4,18,2022-01-29 00:00:00+00:00,346


In [9]:
recency['DaysSinceLastTransaction'].describe()

count    583642.000000
mean        220.832868
std         135.498886
min           1.000000
25%         102.000000
50%         218.000000
75%         337.000000
max         466.000000
Name: DaysSinceLastTransaction, dtype: float64

In [10]:
# calculate R score
recency['R'] = pd.qcut(recency['DaysSinceLastTransaction'],q=qbin,labels=False)
recency.head()

,CustomerID,LatestTransactionDate,DaysSinceLastTransaction,R
0,0,2021-12-24 00:00:00+00:00,382,3
1,1,2022-03-07 00:00:00+00:00,309,2
2,8,2022-07-11 00:00:00+00:00,183,1
3,16,2022-09-16 00:00:00+00:00,116,1
4,18,2022-01-29 00:00:00+00:00,346,3


In [21]:
# frequency df
frequency = pizzahut.groupby('CustomerID')['BillID'].size().reset_index()
frequency.columns = ['CustomerID', 'Frequency']
frequency.head()

,CustomerID,Frequency
0,0,1
1,1,1
2,8,1
3,16,1
4,18,1


In [22]:
# check values of frequency
frequency['Frequency'].describe()

count    583642.000000
mean          1.796606
std           2.269033
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         238.000000
Name: Frequency, dtype: float64

In [23]:
# calculate F score
frequency['F'] = frequency['Frequency'].apply(lambda x: 1 if x == 1 else (3 if x > 1 else (4 if x > 2 else None)))
frequency.head()

,CustomerID,Frequency,F
0,0,1,1
1,1,1,1
2,8,1,1
3,16,1,1
4,18,1,1


In [24]:
# monetary df
monetary = pizzahut.groupby('CustomerID')['SalesAmount'].sum().reset_index()
monetary.head()

,CustomerID,SalesAmount
0,0,411663
1,1,105503
2,8,1098496
3,16,286558
4,18,287492


In [25]:
# check values of money
monetary['SalesAmount'].describe()

count    5.836420e+05
mean     5.477354e+05
std      8.803920e+05
min      7.148000e+03
25%      2.012890e+05
50%      3.118160e+05
75%      5.674812e+05
max      1.337745e+08
Name: SalesAmount, dtype: float64

In [26]:
mq1, mq2, mq3 = monetary['SalesAmount'].describe()[['25%', '50%', '75%']]

In [27]:
# calculate M score
monetary['M'] = monetary['SalesAmount'].apply(lambda x: 1 if x < mq1 else (2 if mq1 <= x < mq2 else (3 if mq2 <= x < mq3 else 4)))
monetary.head()

,CustomerID,SalesAmount,M
0,0,411663,3
1,1,105503,1
2,8,1098496,4
3,16,286558,2
4,18,287492,2


In [28]:
# join 3 df
rfm = pd.merge(recency[['CustomerID', 'R']], frequency[['CustomerID', 'F']], on='CustomerID')
rfm = pd.merge(rfm, monetary[['CustomerID', 'M']], on='CustomerID')
rfm.head()

,CustomerID,R,F,M
0,0,3,1,3
1,1,2,1,1
2,8,1,1,4
3,16,1,1,2
4,18,3,1,2


In [29]:
# concat to RFM column
rfm['RFM'] = pd.concat([rfm['R'], rfm['F'], rfm['M']], axis=1).astype(str).agg(''.join, axis=1)
rfm.head()

,CustomerID,R,F,M,RFM
0,0,3,1,3,313
1,1,2,1,1,211
2,8,1,1,4,114
3,16,1,1,2,112
4,18,3,1,2,312
